In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [2]:
df=pd.read_csv('20240811.export.CSV.zip',compression='zip',header=None,sep='\t')

C:\Users\juan\AppData\Local\Temp\ipykernel_34924\4028254223.py:1: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('20240811.export.CSV.zip',compression='zip',header=None,sep='\t')


In [3]:
# URL where the columns are stored
URL = 'https://www.gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt'

# Making a request to the URL
response = requests.get(URL)

# Checkin if the request was successful
if response.status_code == 200:
    # Get the content of the response
    content = response.content

    # Analise the content of the response
    soup = BeautifulSoup(content, 'html.parser')

    # Get the text of the response and store it in a variable
    columns = soup.get_text()
else:
    print(f'Error: {response.status_code}')

In [4]:
# Split the columns into a list
columns = columns.split()

In [5]:
# Rename the columns of the dataframe
df.columns = columns

In [6]:
df.head(1)

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1192031454,20140814,201408,2014,2014.6137,SWE,SWEDEN,SWE,NaN,NaN,...,-2524279,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.05,-2524279,20240811,https://medafricatimes.com/35885-mali-declares...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69353 entries, 0 to 69352
Data columns (total 58 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GLOBALEVENTID          69353 non-null  int64  
 1   SQLDATE                69353 non-null  int64  
 2   MonthYear              69353 non-null  int64  
 3   Year                   69353 non-null  int64  
 4   FractionDate           69353 non-null  float64
 5   Actor1Code             62407 non-null  object 
 6   Actor1Name             62407 non-null  object 
 7   Actor1CountryCode      39065 non-null  object 
 8   Actor1KnownGroupCode   637 non-null    object 
 9   Actor1EthnicCode       494 non-null    object 
 10  Actor1Religion1Code    1002 non-null   object 
 11  Actor1Religion2Code    226 non-null    object 
 12  Actor1Type1Code        29384 non-null  object 
 13  Actor1Type2Code        1698 non-null   object 
 14  Actor1Type3Code        45 non-null     object 
 15  Ac

# EventID and Date attributes
To start the analysis of this dataset we're going to propose some transformations that may be subject to changes as the project develops.

We keep the GLOBALEVENTID and the SQLDATE, but we normalize the fields related to the date

In [8]:
def day_of_the_year_to_date(year, percentage):

    #Get the date of the first day of the year
    start_date = datetime(year, 1, 1)

    # Calculate the number of days in the year
    days_in_year = 365 if year % 4 != 0 else 366

    # Calculate day number of the year
    day_number = int(days_in_year * (percentage%1))

    # Calculate the target date
    target_date = start_date + timedelta(days=day_number)

    return target_date

In [9]:
# Apply the function to the dataframe[FractionDate] column
new_column = df.apply(lambda row: day_of_the_year_to_date(row['Year'], row['FractionDate']), axis=1)
df.insert(2, 'Date', new_column)


In [10]:
# Drop the columns that are not needed
df.drop(columns=['Year', 'FractionDate', 'MonthYear'], inplace=True)

# ACTOR ATTRIBUTES



In [11]:
# We'll focus on the following columns
df.iloc[:, 3:13].head()

,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code
0,SWE,SWEDEN,SWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWE,SWEDEN,SWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BGD,BANGLADESH,BGD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JORELI,HASSAN,JOR,NaN,NaN,NaN,NaN,ELI,NaN,NaN
4,LEG,PARLIAMENT,NaN,NaN,NaN,NaN,NaN,LEG,NaN,NaN


In [12]:
df.iloc[:, 13:23].head()

,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code
0,AFR,AFRICA,AFR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFR,AFRICA,AFR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JUD,TRIBUNAL,NaN,NaN,NaN,NaN,NaN,JUD,NaN,NaN
3,JUD,TRIBUNAL,NaN,NaN,NaN,NaN,NaN,JUD,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.iloc[:, 23:33].head()

,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone
0,0,60,60,6,2,6.0,10,1,10,0.000000
1,0,70,70,7,2,7.0,10,1,10,0.000000
2,1,111,111,11,3,-2.0,10,5,10,-4.946661
3,1,111,111,11,3,-2.0,85,8,85,-4.675345
4,1,1821,182,18,4,-9.0,20,4,20,-2.383420


In [14]:
df.iloc[:, 33:40].head()

,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID
0,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279
1,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279
2,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683
3,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683
4,4,"Tokyo, Tokyo, Japan",JA,JA40,35.6850,139.7510,-246227


In [15]:
df.iloc[:, 40:47].head()

,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID
0,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279
1,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279
2,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683
3,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683
4,0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.iloc[:, 47:].head()

,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279,20240811,https://medafricatimes.com/35885-mali-declares...
1,4,"Stockholm, Stockholm, Sweden",SW,SW26,59.3333,18.0500,-2524279,20240811,https://medafricatimes.com/35885-mali-declares...
2,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683,20240811,https://www.bignewsnetwork.com/news/274489258/...
3,4,"Dhaka, Dhaka, Bangladesh",BG,BG81,23.7231,90.4086,-2737683,20240811,https://www.bignewsnetwork.com/news/274489258/...
4,4,"Tokyo, Tokyo, Japan",JA,JA40,35.6850,139.7510,-246227,20240811,https://www.theage.com.au/world/asia/rape-surv...
